In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 43.2 MB/s 


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import kerastuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from kerastuner.tuners import Hyperband, BayesianOptimization
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np



<ipython-input-2-719df02fc4dd>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
# df = pd.read_csv('Resources/funding_processed.csv')
df = pd.read_csv('funding_processed.csv')

df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6692,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
3,1,142590,1,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,5000,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34293,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34294,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34295,1,5000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34296,1,5000,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def split_data(data, X, y, seed=5):
    """splits feature and target data into test and train sets and scales feature data

    Args:
        model (class): model class name
        X (array): array containing feature column values
        y (1D array): array containing target variable
        seed (int): random seed value; default=5
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled

def make_model(hp):

    model = Sequential()

    
    hp_activation = hp.Choice('activation', ['relu'])
    
    
    model.add(Dense(units = hp.Int('first_dense', min_value=2, max_value=36),
                    activation = hp_activation,
                    input_dim=36
    ))
    
    for i in range(hp.Int('n_hidden', 1, 2)):
        model.add(Dense(units=hp.Int('dense_' + str(i), min_value=1, max_value=9),
                        activation=hp_activation
    ))
        
        model.add(Dropout(hp.Choice('dropout_' + str(i), [0.0, 0.01, 0.1])
    ))

    
    model.add(Dense(units=1, activation='sigmoid'))
    
    hp_learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2)

    
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate = hp_learning_rate),
                  metrics=['accuracy'])
    
    return model

In [16]:
#split preprocessed data into features and target arrays
X = df.drop(columns=['IS_SUCCESSFUL'])
#target array
y = df['IS_SUCCESSFUL'].values

In [18]:
X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled = split_data(df,X,y)

In [17]:
X_train_scaled.shape

(25723, 36)

In [ ]:
tuner = BayesianOptimization(
    hypermodel = make_model,
    objective='val_accuracy',
    max_trials=30,
    num_initial_points=10,
    seed=5,
    tune_new_entries=True
)

In [20]:
tuner.search(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled,y_test))

Trial 3 Complete [00h 04m 00s]
accuracy: 0.7359561324119568

Best accuracy So Far: 0.7362282872200012
Total elapsed time: 00h 08m 35s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
relu              |relu              |activation
27                |19                |first_dense
1                 |1                 |n_hidden
7                 |2                 |dense_0
0.1               |0                 |dropout_0
0.0012183         |0.0036791         |learning_rate
5                 |None              |dense_1
0.1               |None              |dropout_1

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5911 - accuracy: 0.7035 - val_loss: 0.5669 - val_accuracy: 0.7223
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5677 - accuracy: 0.7228 - val_loss: 0.5635 - val_accuracy: 0.7231
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5613 - accuracy: 0.7245 - val_loss: 

KeyboardInterrupt: ignored

In [ ]:
#pull best hyperparameters
best_hp = tuner.get_best_hyperparameters(1)[0]
best_hp.values


{'activation': 'tanh',
 'first_dense': 9,
 'n_hidden': 2,
 'dense_0': 8,
 'dropout_0': 0.1,
 'dense_1': 4,
 'dropout_1': 0.1}

In [ ]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5557 - accuracy: 0.7315 - 615ms/epoch - 2ms/step
Loss: 0.5556725263595581, Accuracy: 0.7315452098846436


In [ ]:
# Get the top hyperparameters.
best_hps = tuner.get_best_hyperparameters(2)
# Build the model with the best hp.
model = make_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((X_train_scaled, X_test_scaled))
y_all = np.concatenate((y_train, y_test))
model.fit(x=x_all, y=y_all, epochs=100)

In [ ]:
best_model.save('hp_model')
best_model.save('hp_model.h5')
